# Poisson Thompson Sampling

In [ ]:
import numpy as np

from scipy.stats import gamma

import pylab as plt

### Parametros de ajuste y medias de tres productos

Parametros obtenidos de una regresion de Poisson con los datos actuales

In [ ]:
PRODUCTS = {'surrogate': {'alpha': -0.02473518, 'beta': 5.47213038, 'mean': 28.8},
            'extracash': {'alpha': -0.05629794, 'beta': 5.64341742, 'mean': 21.85},
            'plussurrogate': {'alpha': -0.22629794, 'beta': 5.64341742, 'mean': 6.4},
            'test': {'alpha': -7., 'beta': 50., 'mean': 28.8}
            }

### Muestreo de la demanda real

In [ ]:
PROD = 'surrogate'

In [ ]:
def sample_demand(price: float) -> int:
    """
    Esta funcion realiza el muestreo de la distribucion de Poisson 
    con una media igual a la evaluacion de la curva de demanda ajustada previamente.
    
    Este sampleo reemplaza a la obtencion de datos en directo.
    Este metodo puede ser usado como simulador, usando esta funcion,
    o como optimizador en directo introduciendo la demanda actual.
    
    Args:
    price: float, precio ofrecido del producto
    
    Return:
    demand sample: int, demanda estimada para ese precio
    
    PUEDO SALTARME ESTE PASO, EVALUANDO LA FUNCION DE AJUSTE**, AUNQUE INTRODUZCO RANDOMNESS
    
    np.poisson.random -> https://numpy.org/doc/stable/reference/random/generated/numpy.random.poisson.html
    """
    
    global PROD
    
    prod = PRODUCTS[PROD]
    
    demand = np.exp(prod['alpha']*price + prod['beta'])*100
    #demand = prod['alpha']*price + prod['beta']
    
    return np.random.poisson(demand, 1)[0]

In [ ]:
sample_demand(0.1)  

In [ ]:
# **EVALUACION DE LA FUNCION DE AJUSTE

prod = PRODUCTS[PROD]

np.exp(prod['alpha']*0.1*100 + prod['beta'])*100

### Muestreo del modelo Gamma-Poisson

* Parametros iniciales de las distribuciones Gamma
* Sampleo desde el modelo Gamma-Poisson

In [ ]:
# initial params

alpha_0 = 300.     
beta_0 = 1.

PRICES = np.arange(0.1, 1., 0.1)
#PRICES = np.arange(2.49, 5.99, 1)

PRICES = np.round(PRICES, 1)

PARAMS = [{'price': p, 'alpha': alpha_0, 'beta': beta_0} for p in PRICES]

PARAMS

In [ ]:
def sample_demand_from_model(parameters: list) -> list:
    """
    Muestreo del modelo Gamma-Poisson:
    
    Args:
    parameters: list, lista de diccionarios con los parametros de las gammas de cada precio
    
    Return:
    list, lista de parametros actualizados para cada precio
    """
    return list(map(lambda v: np.random.gamma(v['alpha'], 1/v['beta']), parameters))

### Obtencion del precio optimo (precio ofrecido)

In [ ]:
def get_optimal_price(prices: list, demands: list) -> dict:
    """
    Identifica el precio optimo. 
    Nota: este es el modelo Greedy, siempre el argmax.
    Funcion a ser modelada para diferentes algoritmos.
    """
    
    index = np.argmax(prices * demands)
    
    return {'price_index': index, 'price': prices[index]}

### Viz

In [ ]:
def plot_(parameters: list, iteration: int):
    
    x = np.arange(0, 10, 0.20)
    
    for dist in parameters:
        
        y = gamma.pdf(x, a=dist["alpha"], scale= 1/dist["beta"])
        plt.plot(x, y, label = dist["price"])
        plt.xlabel("demand")
        plt.ylabel("pdf")
        
    plt.title(f"PDFs after Iteration: {iteration}")
    plt.legend(loc="upper right")
    plt.show();

## Simulacion

In [ ]:
price_counts = {p: 0 for p in PRICES}

price_counts

In [ ]:
# para N clientes

N = 2000

for t in range(N):

    # muestreo de demanda desde el modelo gamma-poisson
    demands = sample_demand_from_model(PARAMS)

    # obtener precio optimo
    optimal_price_res = get_optimal_price(PRICES, demands)

    # incrementar la cuenta de ese precio
    price_counts[optimal_price_res['price']] += 1

    # ofrecer precio optimo y OBSERVAR demanda
    true_demand = sample_demand(optimal_price_res['price'])

    # actualizar parametros del modelo / actualizar creencia
    v = PARAMS[optimal_price_res['price_index']] # para ese precio...
    v['alpha'] += true_demand                     # alfa + demanda
    v['beta'] += 1                               # beta + 1
    
    
    if t%200 == 0:
        plot_(PARAMS, t)

In [ ]:
price_counts

In [ ]:
PARAMS

In [ ]:
demands

In [ ]:
optimal_price_res

In [ ]:
true_demand

In [ ]:
v